# Training using PyTorch with SMMP

In [158]:
install_needed = True  # should only be True once
# install_needed = False

In [159]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip 
    !{sys.executable} -m pip install -U sagemaker smdebug ipywidgets --user 
    !{sys.executable} -m pip install -U torchvision split-folders wget
    !/bin/bash ./local/local_change_setting.sh
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Add data-root and default-shm-size=10G
Redirecting to /bin/systemctl restart docker.service
Docker Restart


In [1]:
import sagemaker
from sagemaker.debugger import (
    Rule, ProfilerRule, rule_configs, ProfilerConfig, 
    FrameworkProfile, DetailedProfilingConfig, 
    DataloaderProfilingConfig, PythonProfilingConfig)
import time
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [2]:
sagemaker.__version__

'2.94.0'

## Data
### Getting the data



### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [3]:
# import wget
# import tarfile
# import shutil
# import os
# import glob
# import splitfolders

# def make_dir(img_path, delete=True):
#     import shutil, os
#     try:
#         if not os.path.exists(img_path):
#             os.makedirs(img_path)
#         else:
#             if delete:
#                 shutil.rmtree(img_path)
#     except OSError:
#         print("Error")

# rawimg_path = 'raw_img'
# output_path = 'data'
# dataset_path = './dataset'

# make_dir(rawimg_path)
# make_dir(dataset_path)

# if not (os.path.isfile("images.tar.gz") and tarfile.is_tarfile("images.tar.gz")):
#     wget.download('https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz')
# tar = tarfile.open("images.tar.gz")
# tar.extractall(path=rawimg_path)
# tar.close()

# import cv2
# import numpy as np

# def checkImage(path):
# #     print(path)
#     try:
#         with open(path, 'rb') as f:
#             data = f.read()
#             f.seek(-2,2)
#             value = f.read()

#         encoded_img = np.frombuffer(data, dtype = np.uint8)
#         img_cv = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
# #         print(img_cv)
#         if img_cv.shape[0]>0 and value == b'\xff\xd9':
#             return True
#     except:
#         return False

# corrupt_img = ['Egyptian_Mau_14.jpg','Egyptian_Mau_139.jpg','Egyptian_Mau_145.jpg','Egyptian_Mau_156.jpg',
#                'Egyptian_Mau_167.jpg','Egyptian_Mau_177.jpg','Egyptian_Mau_186.jpg','Egyptian_Mau_191.jpg',
#                'Abyssinian_5.jpg','Abyssinian_34.jpg','chihuahua_121.jpg','beagle_116.jpg']

# file_dir = os.path.join(rawimg_path, 'images')
# output_dir = os.path.join(rawimg_path, output_path)

# for file_path in glob.glob(file_dir + "/*"):
#     filename = file_path.split("/")[2]
#     if checkImage(file_path) and filename not in corrupt_img:
#         dir_name = filename.split("_")
#         dir_name.pop()
#         dir_name = "_".join(dir_name)
#         dir_path = os.path.join(output_dir, dir_name)
#         make_dir(dir_path, False)
#         target_path = os.path.join(dir_path, filename)
#         shutil.copyfile(file_path, target_path)
#     else:
#         print(file_path)

In [4]:
# num = 0
# for file_path in glob.glob(output_dir + "/*/*"):
# #     print(file_path)
#     if not checkImage(file_path):
# #         print(file_path)
#         num += 1
# print(num)

In [5]:
# splitfolders.ratio(output_dir, output=dataset_path, seed=1337, ratio=(.8, .1, .1)) # default values
# inputs = 's3://{}/{}'.format(bucket, 'oxford_pet_dataset')
# !aws s3 rm $inputs --quiet --recursive
# !aws s3 cp ./dataset $inputs --quiet --recursive

In [6]:
inputs

NameError: name 'inputs' is not defined

## Train
### Training script
The script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [7]:
# !pygmentize ./train_code/pytorch_mnist_smdp.py

In [8]:
hyperparameters = {
    'epochs': 1,
    'batch_size':1,
    'img_size':3300, #10000,
    'ddp' : True,
    'smp' : True
}

In [9]:
distribution={}

if hyperparameters['smp']:
    distribution_hyper = {
        "ddp": hyperparameters['ddp'],
#         "tensor_parallel_degree": 4,
        "partitions": 8,  ## pipeline_parallel_degree
    #     "shard_optimizer_state": True,
        "prescaled_batch": False,
        "fp16_params": True,
        "optimize": "memory", #"speed",
        "auto_partition": True,
    #     "default_partition": 0,
        "microbatches":1
    }
    
    hyperparameters['prescaled_batch'] = distribution_hyper['prescaled_batch']
    
    distribution['smdistributed'] = {
        "modelparallel": {
            "enabled":True,
            "parameters": distribution_hyper
        }   
    }
    mpioptions = "-x NCCL_DEBUG=WARN -x SMDEBUG_LOG_LEVEL=ERROR "
    mpioptions += "-x SMP_DISABLE_D2D=1 -x SMP_D2D_GPU_BUFFER_SIZE_BYTES=1 -x SMP_NCCL_THROTTLE_LIMIT=1 "
    mpioptions += "-x FI_EFA_USE_DEVICE_RDMA=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1"


    distribution["mpi"]={
        "enabled": True,
        "processes_per_host": 8, # Pick your processes_per_host
        "custom_mpi_options": mpioptions      
    }
else:
    distribution = {
        "mpi": {
            "enabled": True
        }
    }

In [10]:
# distribution = {"smdistributed": {
#                     "dataparallel": {
#                             "enabled": True
#                     }
#                }
#              }

In [11]:
# distribution = {"mpi": {
#                     "enabled": True
#                }
#              }

In [12]:
instance_type = 'ml.p4d.24xlarge'
instance_type = 'local_gpu'
instance_count = 1
entry_point = 'main.py'

In [13]:
!sudo rm -rf /home/ec2-user/SageMaker/temp
!mkdir /home/ec2-user/SageMaker/temp
!sudo rm -rf /tmp/*

In [14]:
if instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    from pathlib import Path

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}, 'container_root':'/home/ec2-user/SageMaker/temp'}
    s3_data_path = f'file://{Path.cwd()}/dataset'
    source_dir = f'{Path.cwd()}/code'
    checkpoint_s3_bucket = None
else:
    sess = boto3.Session()
    sagemaker_session = sagemaker.Session()
    sm = sess.client('sagemaker')
    s3_data_path = inputs
    source_dir = 'code'
    checkpoint_s3_bucket = f's3://{bucket}/checkpoints'

In [15]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point=entry_point,
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.10',
                    py_version='py38',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    sagemaker_session=sagemaker_session,
                    disable_profiler=True,
                    debugger_hook_config=False,
                    max_run=1*30*60,
                    hyperparameters=hyperparameters
                   )

In [16]:
!sudo rm -rf code/core.*

In [ ]:
job_name = "training-job-{}".format(int(time.time()))

estimator.fit({'training': s3_data_path},
              job_name=job_name,
              wait=False
            )

Creating wtz4xle41m-algo-1-vtt7o ... 
Creating wtz4xle41m-algo-1-vtt7o ... done
Attaching to wtz4xle41m-algo-1-vtt7o
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:58,927 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,330 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,334 sagemaker_pytorch_container.training INFO     Invoking user training script.
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,493 sagemaker-training-toolkit INFO     Starting MPI run as worker node.
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,493 sagemaker-training-toolkit INFO     Creating SSH daemon.
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,495 sagemaker-training-toolkit INFO     Waiting for MPI workers to establish their SSH connections
wtz4xle41m-algo-1-vtt7o | 2022-06-11 08:27:59,496 sagemaker-training-toolkit INFO     Env Hosts: ['algo-1

In [ ]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)